In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from functionst import * 
import numpy as np
%matplotlib inline

In [3]:
# Dataset1: Client Profiles: Demographics like age, gender, and account details of our clients.

In [4]:
file_path='data/df_final_demo.txt'

df_clients = pd.read_csv(file_path, delimiter=',')
df_clients.head()

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6.0,73.0,60.5,U,2.0,45105.30,6.0,9.0
1,2304905,7.0,94.0,58.0,U,2.0,110860.30,6.0,9.0
2,1439522,5.0,64.0,32.0,U,2.0,52467.79,6.0,9.0
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0


In [5]:
# Dataset2: Digital Footprints pt1 & pt2: 
# A detailed trace of client interactions online, divided into two parts: pt_1 and pt_2. 
# It’s recommended to merge these two files prior to a comprehensive data analysis.
file_path='data/df_final_web_data_pt_1.txt'
df2 = pd.read_csv(file_path, delimiter=',')
file_path = "Data/df_final_web_data_pt_1.txt"
df3 = pd.read_csv(file_path, delimiter=',')
df_concat = pd.concat([df2, df3])  # Concatenated 

In [6]:
#Dataset 3: Experiment Roster: A list revealing which clients were part of the grand experiment.
file_path='data/df_final_experiment_clients.txt'
df_roster = pd.read_csv(file_path, delimiter=',')
df_roster.head()

,client_id,Variation
0,9988021,Test
1,8320017,Test
2,4033851,Control
3,1982004,Test
4,9294070,Control


Cleaning df_Clients

Check for Duplicates

In [7]:
df_clients.duplicated().sum() # Check

0

In [8]:
# Rename columns
df_clients.rename(columns={"clnt_tenure_yr": "client_tenure_yr",
                           "clnt_tenure_mnth":"client_tenure_mnth",
                           "clnt_age":"client_age",
                           "gendr":"gender",
                           "bal":"balance"
                           }, inplace=True)

In [ ]:
# Checking for NaN Rows
nan_rows = df_clients[df_clients.isna().any(axis=1)]
# Drop NaN Values with more many NaNs
df_clients.dropna(thresh=4,inplace=True)
# Fill NaN in client age
df_clients["client_age"].fillna(df_clients["client_age"].mean(),inplace=True)

# Drop NaN Values with more many NaNs 
df_clients.dropna(thresh=4,inplace=True)

# Fill NaN in client age
df_clients["client_age"].fillna(df_clients["client_age"].mean(),inplace=True)

# Filled the Client tenure yr with NaN when Client age < then tenure
df_clients.loc[df_clients["client_tenure_yr"] > df_clients["client_age"], "client_tenure_yr"] = np.nan

In [10]:
# Defining age Group and Assigning to client Data

df_clients["client_age_group"] = df_clients["client_age"].apply(age_group)

Clean df_concat

In [11]:
# Transformed DATETIME
df_concat["date_time"] = pd.to_datetime(df_concat["date_time"])

In [12]:
df_concat.duplicated().sum()

345236

Joined Clients and Joined DC_Concat with Roster

In [28]:
# Joined Clients with the Roster an dropped the NaN Values
df_client_joined = pd.merge(df_clients,df_roster, how="left", on="client_id")

In [14]:
# Joined DF Concat with the Roster and dropped the duplicates
df_concat_joined = pd.merge(df_concat,df_roster, on="client_id", how="left")
df_concat_joined.dropna(subset="Variation",inplace=True)
df_concat_joined.drop_duplicates(inplace=True)

In [40]:
# Add a column if the client finished the process

# Identify the client_ids that have a "finished" step
clients_with_finished = df_concat_joined[df_concat_joined['process_step'] == 'confirm']['client_id']

# Using map to replace boolean values
df_client_joined['finished'] = df_client_joined["client_id"].isin(clients_with_finished).map({True:"yes",False:"no"})

df_client_joined["finished"].unique()

array(['yes', 'no'], dtype=object)

Who are the clients

In [ ]:
# Who are the clients? By Age
most_clients = df_clients.groupby("client_age_group")["logons_6_mnth"].sum().sort_values( ascending=False)
most_clients

In [ ]:
# Checking 
sns.barplot(data= most_clients)
plt.title("age groups by age")
plt.xlabel("age groups")
plt.ylabel("logins last 6 month")

In [27]:
# Who are the clients by tenure

In [ ]:
sns.boxplot(data=df_clients["client_tenure_yr"])

In [ ]:
sns.violinplot(data=df_clients["client_tenure_yr"])

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(df_clients['client_age'], df_clients['client_tenure_yr'], color='blue', alpha=0.7)
plt.xlabel('Client Age', fontsize=12)
plt.ylabel('Client Tenure (Years)', fontsize=12)
plt.title('Client Age vs. Client Tenure (Years)', fontsize=15)

Sorting df_concat and trying to calculate time of steps

In [ ]:
# Df_concat Sorted and time_diff_in_minutes

# With this we are able to analyze
df_concat_joined_sorted = df_concat_joined.sort_values(by=['client_id', 'visit_id', 'date_time'])
df_concat_joined_sorted['time_diff_in_minutes'] = df_concat_joined_sorted.groupby(['client_id', 'visit_id'])['date_time'].diff().dt.total_seconds() / 60
df_concat_joined_sorted



In [ ]:
# How many reached where
df_concat_joined_sorted["process_step"].value_counts()
